In [1]:
########## import libraries ############
import numpy as np
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros
import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import MaxPooling1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
import IPython; from IPython.display import display, HTML
def dfPrint(df):
    display(HTML(df.to_html()))
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cardioid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [2]:
# ############# import data ##############
# def import_data():
#     train_path = "../data/text_large_infy_mod.csv"
#     data = pd.read_csv(train_path)
#     data.reset_index(inplace=True)
#     data.drop('index', axis=1, inplace=True)
#     data.drop(['year','month','date','hour','mins','no'], axis=1, inplace=True)
#     print ('dataset loaded with shape', data.shape)
#     return data

# data = import_data()
# dfPrint(data.sample(5))

In [57]:
 ############ import data ##############
def import_data():
    train_path = "../data/AXISBANK.csv"
    data = pd.read_csv(train_path)
#     data.reset_index(inplace=True)
#     data.drop('index', axis=1, inplace=True)
    data.drop(['time','index'], axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)
    return data

data = import_data()
quantity = 1
dfPrint(data.sample(20))

dataset loaded with shape (1492, 2)


,title,label
342,Google Alert - currency notes rbi,1
430,Many countries with similar macroeconomic envi...,1
423,"ICICI Bank's Chanda Kochhar, Axis Bank's Sikha...",1
697,Today Stock News – 01.03.2018,1
828,"Sensex Opens Higher, Nifty Reclaims 10,600 | T...",0
664,Market Now: These stocks jump over 7% in a bea...,1
1449,"PNB fraud: Two rogue employees, one famed jewe...",1
820,"Sensex And Nifty Upbeat Amid Global Rally, PNB...",0
690,"SBI raises retail FD rates, first time in thre...",1
928,Daiichi: Unpledged assets of Singh brothers to...,1


In [11]:
def clean_decription(line):
    words = list(set(line.split()))
    words = list(sorted([x.lower() for x in words]))
#     print (len(words))
#     print (words)
    words2 = words.copy()
    for word in list(set(words2).intersection(stop_words)):
        words.remove(word)
#     out = ' '.join(words)
    out = ' '.join(e for e in words if e.isalnum())
    if out == "":
        print(words)
#     table = str.maketrans({key: None for key in string.punctuation})
#     out.translate(table, string.punctuation)
    return out
data["title"] = data["title"].map(clean_decription)

In [12]:
############### define documents ################
docs = data["title"]+
############# define class labels ################
labels = data["label"]
############## prepare tokenizer #################
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

########## integer encode the documents ##########
encoded_docs = t.texts_to_sequences(docs)
########## pad documents to a max length ##########
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

print(docs.sample(5))
print(vocab_size)

1247    banking indian lender news pnb slides stories ...
229                        account bank reflecting refund
665                                         atm cash indi
38                           axis bank pay start whatsapp
885     cbi details five nostro pnb seeks transactions...
Name: title, dtype: object
2328


In [13]:
### load the whole embedding into memory ############
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for num, line in enumerate(f):
    values = line.split()
    word = values[0]
    if word in stop_words :
#         print (word)
        continue
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()
max_width = 300
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 399851 word vectors.


In [14]:
########### create a weight matrix for words in training docs ############
embedding_matrix = zeros((vocab_size, max_width))
mil=0
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        mil+=1
        embedding_matrix[i] = embedding_vector
print(mil)

2148


In [ ]:
# ################ define model#################
# model = Sequential()
# e = Embedding(vocab_size, max_width, weights=[embedding_matrix], input_length=max_length, trainable=True)
# model.add(e)
# model.add(LSTM(35,return_sequences=False))
# # model.add(GlobalMaxPooling1D(pool_size=2, strides=None, padding='valid'))
# # model.add(Flatten())
# model.add(Dense(50,  kernel_initializer="normal",activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(Dense(25,  kernel_initializer="normal",activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(Dense(15,  kernel_initializer="normal",activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))

# ############## compile the model ##############
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc','binary_accuracy'])
# ############# summarize the model ##############
# print(model.summary())

In [15]:
################ define model#################
model = Sequential()
e = Embedding(vocab_size, max_width, weights=[embedding_matrix], input_length=max_length, trainable=True)
model.add(e)
model.add(Bidirectional(LSTM(35, return_sequences=True)))
model.add(GlobalMaxPooling1D())
# model.add(Bidirectional(LSTM(10)))
# model.add(Flatten())
model.add(Dense(15,  kernel_initializer="normal",activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

############## compile the model ##############
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc','binary_accuracy'])
############# summarize the model ##############
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          698400    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 70)           94080     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 70)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                1065      
_________________________________________________________________
dropout_1 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 793,561
Trainable params: 793,561
Non-trainable params: 0
_________________________________________________________________
None

In [16]:
############### split into train test data #################
padded_docs_train, padded_docs_test, labels_train, labels_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=69)
################### fit the model ##########3#############
model.fit(padded_docs_train, labels_train, epochs=15, validation_split = 0.1,verbose=1)

Train on 1073 samples, validate on 120 samples
Epoch 1/15
1073/1073 [==============================] - 8s 7ms/step - loss: 0.6920 - acc: 0.5349 - binary_accuracy: 0.5349 - val_loss: 0.6809 - val_acc: 0.5833 - val_binary_accuracy: 0.5833
Epoch 2/15
1073/1073 [==============================] - 6s 6ms/step - loss: 0.6778 - acc: 0.5722 - binary_accuracy: 0.5722 - val_loss: 0.6744 - val_acc: 0.6167 - val_binary_accuracy: 0.6167
Epoch 3/15
1073/1073 [==============================] - 7s 6ms/step - loss: 0.6610 - acc: 0.6076 - binary_accuracy: 0.6076 - val_loss: 0.6568 - val_acc: 0.6500 - val_binary_accuracy: 0.6500
Epoch 4/15
1073/1073 [==============================] - 6s 6ms/step - loss: 0.5933 - acc: 0.6943 - binary_accuracy: 0.6943 - val_loss: 0.6305 - val_acc: 0.6417 - val_binary_accuracy: 0.6417
Epoch 5/15
1073/1073 [==============================] - 6s 6ms/step - loss: 0.4970 - acc: 0.7642 - binary_accuracy: 0.7642 - val_loss: 0.6018 - val_acc: 0.6750 - val_binary_accuracy: 0.6750
Epo

In [17]:
# evaluate the model
loss, accuracy,binaryAccuracy = model.evaluate(padded_docs_test, labels_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))
print('binaryAccuracy: %f' % (binaryAccuracy*100))


299/299 [==============================] - 1s 2ms/step
Accuracy: 67.558528
binaryAccuracy: 67.558528


In [58]:
predict_path = "../data/AXISBANKTest.csv"
test = pd.read_csv(predict_path)
############## prepare tokenizer #################
t1 = Tokenizer()
t1.fit_on_texts(test)

# predict_path = "../data/AXISBANKTest.csv"
# test = pd.read_csv(predict_path)
print(test.size)
########## integer encode the documents ##########
encoded_test = t.texts_to_sequences(test['title'])
print(len(encoded_test))
########## pad documents to a max length ##########
max_length = 100
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

predicted = model.predict(padded_test)
# for i in range(predicted.size):
#     if predicted[i] < 0.5 :
#         predicted[i] = 0.0
#     else :
#         predicted[i] = 1.0

38
19


In [68]:
confidance = np.mean(predicted)
if confidance > 0.5:
    print ("buy")
    print("confidance is :",confidance)
    
else:
    if quantity != 0 :
        print ("sell")
        print("confidance is :",1-confidance)

buy
confidance is : 0.631579
